## data.ipynb

### data for training

In [123]:
# import libraries
import FinanceDataReader as fdr
import pandas as pd
import numpy as np

In [124]:
# 데이터 수집 기간 설정
start_date = '20200106'
end_date = '20220429'

In [125]:
# function for columns renaming
def cols_rename(data_set, target_name):
    for i in data_set.columns:
        if i == 'Date':
            pass
        else:
            data_set.rename(columns={i:target_name+'_'+i}, inplace=True)
    return data_set

In [126]:
# function for column renaming
def col_rename(data_set, name_from, name_to):
    for i in data_set.columns:
        if i != name_from:
            pass
        else:
            data_set.rename(columns={i:name_to}, inplace=True)
    return data_set
    

In [127]:
# function for adjusting friday to sunday data as new friday data
def weekendToFriday(target, idx, col, returnType): # idx: Friday
    if (returnType == 'intType'):
        target.loc[idx, col] = (target.loc[idx, col]//7)*1 + (target.loc[idx+1, col]//7)*2 + (target.loc[idx+2, col]//7)*4
    elif (returnType == 'floatType'):
        target.loc[idx, col] = (target.loc[idx, col]/7)*1 + (target.loc[idx+1, col]/7)*2 + (target.loc[idx+2, col]/7)*4
    return target.loc[idx, col]

In [128]:
# make business_days: 주말 미포함
Business_days = pd.DataFrame(pd.date_range(start_date, end_date, freq='B'), columns = ['Date'])
Business_days['weekday'] = Business_days.Date.apply(lambda x: x.weekday())
Business_days['weeknum'] = Business_days.Date.apply(lambda x: x.strftime('%V'))

### 주말 미포함 데이터

In [129]:
# KOSPI 추가
KOSPI = fdr.DataReader('KS11', start_date, end_date).reset_index()
KOSPI.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(KOSPI, 'kospi')
data = pd.merge(Business_days, KOSPI, how='outer')

In [130]:
# KOSDAQ 추가
KOSDAQ = fdr.DataReader('KQ11', start_date, end_date).reset_index()
KOSDAQ.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(KOSDAQ, 'kosdaq')
data = pd.merge(data, KOSDAQ, how='outer')

In [131]:
# 미국증시: 나스닥(NASDAQ) 추가
NAS = fdr.DataReader('NASDAQCOM', start_date, end_date, data_source='fred').reset_index()
col_rename(NAS, 'DATE', 'Date')
col_rename(NAS, 'NASDAQCOM', 'nasdaq_Close')
data = pd.merge(data, NAS, how='outer')

In [132]:
# 환율: 원달러(USD) 추가
USD = fdr.DataReader('USD/KRW', start_date, end_date).reset_index()
USD.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(USD, 'usd')
data = pd.merge(data, USD, how='outer')

In [133]:
# 환율: 원엔(JPY) 추가
JPY = fdr.DataReader('JPY/KRW', start_date, end_date).reset_index()
JPY.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(JPY, 'jpy')
data = pd.merge(data, JPY, how='outer')

In [134]:
# 환율: 호주달러/스위스프랑 추가
ACF = fdr.DataReader('AUD/CHF', start_date, end_date).reset_index()
ACF.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(ACF, 'acf')
data = pd.merge(data, ACF, how='outer')

In [135]:
# 국채: 미 국채 10년 추가
UGB = fdr.DataReader('DGS10', start_date, end_date, data_source='fred').reset_index()
col_rename(UGB, 'DATE', 'Date')
col_rename(UGB, 'DGS10', 'ugb_Close')
data = pd.merge(data, UGB, how='outer')

In [136]:
# 변동성 지수 추가
VIX = fdr.DataReader('VIXCLS', start_date, end_date, data_source='fred').reset_index()
col_rename(VIX, 'DATE', 'Date')
col_rename(VIX, 'VIXCLS', 'vix_Close')
data = pd.merge(data, VIX, how='outer')

### 주말 포함 데이터

In [137]:
# 암호화폐: 비트코인(BitCoin) 추가
BTC = fdr.DataReader('BTC/KRW',start_date,end_date).reset_index()
BTC['dayofweek'] = BTC['Date'].dt.dayofweek # 요일 (월 = 0)
BTC.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(BTC, 'btc')

# 예외처리: 마지막 데이터가 토요일인 경우
if (BTC.loc[len(BTC)-1, 'btc_dayofweek'] == 5):
    BTC.drop(len(BTC)-1, inplace=True)

# 예외처리: 첫 데이터가 주말인 경우
if (BTC.loc[0, 'btc_dayofweek'] == 6):
    BTC.drop(0, inplace=True)
elif (BTC.loc[0, 'btc_dayofweek'] == 5):
    BTC.drop(0, inplace=True)
    BTC.drop(1, inplace=True)

# 금토일 데이터를 금요일 데이터로 병합 후 주말 데이터 제거
# 금토일 데이터 가중치 >> 1:2:4
for idx in BTC.index:
    if (BTC.loc[idx, 'btc_dayofweek'] == 6):
        BTC.loc[idx-2, 'btc_Close'] = weekendToFriday(BTC, idx-2, 'btc_Close', 'intType')
        BTC.loc[idx-2, 'btc_Volume'] = weekendToFriday(BTC, idx-2, 'btc_Volume', 'floatType')
        BTC.loc[idx-2, 'btc_Change'] = np.ceil(weekendToFriday(BTC, idx-2, 'btc_Change', 'floatType')*1000)/1000
for idx in BTC.index:
    if (BTC.loc[idx, 'btc_dayofweek'] == 5 or BTC.loc[idx, 'btc_dayofweek'] == 6):
        BTC.drop(idx, inplace=True)

# 데이터 가공
BTC.drop(['btc_dayofweek'], axis=1, inplace=True)
BTC.reset_index(inplace=True)
BTC.drop(['index'], axis=1, inplace=True)

data = pd.merge(data, BTC, how='outer')

In [138]:
# 국채: 한국 국채 추가
KGB = fdr.DataReader('KR10YT=RR', start_date, end_date).reset_index()
KGB['dayofweek'] = KGB['Date'].dt.dayofweek # 요일 (월 = 0)
KGB.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(KGB, 'kgb')

# 예외처리: 마지막 데이터가 토요일인 경우
if (KGB.loc[len(KGB)-1, 'kgb_dayofweek'] == 5):
    KGB.drop(len(KGB)-1, inplace=True)

# 예외처리: 첫 데이터가 주말인 경우
if (KGB.loc[0, 'kgb_dayofweek'] == 6):
    KGB.drop(0, inplace=True)
elif (KGB.loc[0, 'kgb_dayofweek'] == 5):
    KGB.drop(0, inplace=True)
    KGB.drop(1, inplace=True)

# 금토일 데이터를 금요일 데이터로 병합 후 주말 데이터 제거
# 금토일 데이터 가중치 >> 1:2:4
for idx in KGB.index:
    if (KGB.loc[idx, 'kgb_dayofweek'] == 6):
        KGB.loc[idx-2, 'kgb_Close'] = weekendToFriday(KGB, idx-2, 'kgb_Close', 'floatType')
        KGB.loc[idx-2, 'kgb_Change'] = weekendToFriday(KGB, idx-2, 'kgb_Change', 'floatType')
        KGB.loc[idx-2, 'kgb_Change'] = np.ceil(weekendToFriday(KGB, idx-2, 'kgb_Change', 'floatType')*1000)/1000
for idx in KGB.index:
    if (KGB.loc[idx, 'kgb_dayofweek'] == 5 or KGB.loc[idx, 'kgb_dayofweek'] == 6):
        KGB.drop(idx, inplace=True)

# 데이터 가공
KGB.drop(['kgb_dayofweek'], axis=1, inplace=True)
KGB.reset_index(inplace=True)
KGB.drop(['index'], axis=1, inplace=True)

data = pd.merge(data, KGB, how='outer')

In [139]:
# NaN 값은 전일 값으로 대체. 
data = data.fillna(method='ffill')
data = data.fillna(method='bfill') # 예외처리
print(data)

          Date  weekday weeknum  kospi_Close  kospi_Volume  kospi_Change  \
0   2020-01-06        0      02      2155.07  5.926700e+08       -0.0098   
1   2020-01-07        1      02      2175.54  5.682400e+08        0.0095   
2   2020-01-08        2      02      2151.31  9.138300e+08       -0.0111   
3   2020-01-09        3      02      2186.45  5.926000e+08        0.0163   
4   2020-01-10        4      02      2206.39  5.945400e+08        0.0091   
..         ...      ...     ...          ...           ...           ...   
600 2022-04-25        0      17      2657.13  9.883500e+05       -0.0176   
601 2022-04-26        1      17      2668.31  8.899400e+08        0.0042   
602 2022-04-27        2      17      2639.06  1.070000e+09       -0.0110   
603 2022-04-28        3      17      2667.49  8.991700e+08        0.0108   
604 2022-04-29        4      17      2695.05  9.624000e+05        0.0103   

     kosdaq_Close  kosdaq_Volume  kosdaq_Change  nasdaq_Close  ...  \
0          655.31

### 매주 금요일 데이터

In [140]:
# 금융스트레스 지수 추가
#FSI = fdr.DataReader('STLFSI2', start_date, end_date, data_source='fred').reset_index()
#col_rename(FSI, 'DATE', 'Date')
#col_rename(FSI, 'STLFSI2', 'fsi_Close')
#data = pd.merge(data, FSI, how='outer')

# 사용하지 않기로 결정

### 데이터가 더이상 제공되지 않음 (~22/1/21)

In [141]:
# 신용경색 지수 추가
#TED = fdr.DataReader('TEDRATE', start_date, end_date, data_source='fred').reset_index()
#col_rename(TED, 'DATE', 'Date')
#col_rename(TED, 'TEDRATE', 'ted_Close')
#data = pd.merge(data, TED, how='outer')

# 사용하지 않기로 결정